## Packages

In [1]:
# run these commands once at the start of working on this notebook
# they make sure that any imported modules are reloaded before executing
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import folium as fm
import geopandas as gpd
from shapely.geometry import Point, Polygon
from fiona.drvsupport import supported_drivers
import panel as pn
import re
import matplotlib.pyplot as plt
pn.extension(sizing_mode="stretch_width")


## Data Sources

In [3]:
supported_drivers['KML'] = 'rw'
project_zips = gpd.read_file('./Data/Geographies/DC_MD_VA_Zipcodes.kml', driver='KML')

In [4]:
def get_zip_code(cell):
    match = re.search(r'<at><openparen>([0-9]+)<closeparen>', cell)
    if match:
        return match.group(1)
zips = list()
for r in project_zips.iterrows():
    zips.append(get_zip_code(r[1]['Name']))
project_zips['Zip_Code'] = zips
project_zips.head()

,Name,Description,geometry,Zip_Code
0,<at><openparen>20001<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-77.02758 38.90964 0.00000, -77.02...",20001
1,<at><openparen>20002<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-77.01217 38.89209 0.00000, -77.01...",20002
2,<at><openparen>20003<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-77.01402 38.88236 0.00000, -77.01...",20003
3,<at><openparen>20004<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-77.03365 38.89735 0.00000, -77.03...",20004
4,<at><openparen>20005<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-77.03654 38.90252 0.00000, -77.03...",20005


## Build dataframes

### BAH_df

In [5]:
#1. Dataframe of zip codes and MHAs (sorted_zipmha23.txt)
sorted_zipmha23_df = pd.read_csv('Data/sorted_zipmha23.txt', sep=" ", header=None, names=["zip_code", "mha"], dtype={"zip_code":"string"})
#2. Dataframe of MHAs and MHA names (mhanames23.txt)
mhanames23_df = pd.read_csv('Data/mhanames23.txt', sep=";", header=None, names=["mha", "mha_names"]) 
#2a. Combine 1st two df on mha
zip_mha_names_df = pd.merge(sorted_zipmha23_df, mhanames23_df, on='mha') 
#3. 2023 BAH rates without dependents (bahwo23.txt)
dbahwo23_df = pd.read_csv('Data/bahwo23.txt', sep=',', names=['mha', 
                                      'EO1', 'EO2', 'EO3', 'EO4', 'EO5', 'EO6', 'EO7', 'EO8', 'EO9',
                                      'WO1','WO2','WO3','WO4','WO5', 'O01E', 'O02E', 'O03E',
                                      'O01','O02','O03','O04','O05','O06','O07','O08','O09', 'O010', 'dep_status'])
dbahwo23_df['dep_status']='WO'
#4. 2023 BAH rates with dependents (bahw23.txt):
bahw23_df = pd.read_csv('Data/bahw23.txt', sep=',', names=['mha', 
                                      'EO1', 'EO2', 'EO3', 'EO4', 'EO5', 'EO6', 'EO7', 'EO8', 'EO9',
                                      'WO1','WO2','WO3','WO4','WO5', 'O01E', 'O02E', 'O03E',
                                      'O01','O02','O03','O04','O05','O06','O07','O08','O09', 'O010', 'dep_status'])
bahw23_df['dep_status']='W'
#5. Stack BAH WO & W dfs from #3 and #4
bahtot23_df = pd.concat([dbahwo23_df,bahw23_df])
#6. Combine BAH rates and zipcodes
BAH_df = pd.merge(zip_mha_names_df, bahtot23_df, on='mha', how='left') 
BAH_df

,zip_code,mha,mha_names,EO1,EO2,EO3,EO4,EO5,EO6,EO7,...,O02,O03,O04,O05,O06,O07,O08,O09,O010,dep_status
0,00501,NY218,"LONG ISLAND, NY",2733.0,2733.0,2733.0,2733.0,2928.0,3105.0,3390.0,...,3597.0,3984.0,4227.0,4284.0,4368.0,4443.0,4443.0,4443.0,4443.0,WO
1,00501,NY218,"LONG ISLAND, NY",3378.0,3378.0,3378.0,3378.0,3708.0,4140.0,4230.0,...,4137.0,4383.0,4980.0,5412.0,5460.0,5508.0,5508.0,5508.0,5508.0,W
2,00544,NY218,"LONG ISLAND, NY",2733.0,2733.0,2733.0,2733.0,2928.0,3105.0,3390.0,...,3597.0,3984.0,4227.0,4284.0,4368.0,4443.0,4443.0,4443.0,4443.0,WO
3,00544,NY218,"LONG ISLAND, NY",3378.0,3378.0,3378.0,3378.0,3708.0,4140.0,4230.0,...,4137.0,4383.0,4980.0,5412.0,5460.0,5508.0,5508.0,5508.0,5508.0,W
4,06390,NY218,"LONG ISLAND, NY",2733.0,2733.0,2733.0,2733.0,2928.0,3105.0,3390.0,...,3597.0,3984.0,4227.0,4284.0,4368.0,4443.0,4443.0,4443.0,4443.0,WO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81898,99928,AK400,"KETCHIKAN, AK",2034.0,2034.0,2034.0,2034.0,2454.0,2472.0,2532.0,...,2469.0,2685.0,3057.0,3324.0,3351.0,3378.0,3378.0,3378.0,3378.0,W
81899,99929,AK400,"KETCHIKAN, AK",1527.0,1527.0,1527.0,1527.0,1842.0,1854.0,1899.0,...,1974.0,2289.0,2529.0,2586.0,2670.0,2712.0,2712.0,2712.0,2712.0,WO
81900,99929,AK400,"KETCHIKAN, AK",2034.0,2034.0,2034.0,2034.0,2454.0,2472.0,2532.0,...,2469.0,2685.0,3057.0,3324.0,3351.0,3378.0,3378.0,3378.0,3378.0,W
81901,99950,AK400,"KETCHIKAN, AK",1527.0,1527.0,1527.0,1527.0,1842.0,1854.0,1899.0,...,1974.0,2289.0,2529.0,2586.0,2670.0,2712.0,2712.0,2712.0,2712.0,WO


### FMR_df

In [6]:
#1. Dataframe of Fair Market Rents by metro areas (source: fy2023_safmrs.xlsx)
FMR_df = pd.read_excel('Data/fy2023_safmrs.xlsx', dtype=str)
FMR_df = FMR_df[['ZIP\nCode',
                 'SAFMR\n1BR',
                 'SAFMR\n2BR',
                 'SAFMR\n3BR',
                 'SAFMR\n4BR',
                 'HUD Area Code',
                 'HUD Metro Fair Market Rent Area Name']]
FMR_df.rename(columns={'ZIP\nCode':'zip_code',
                       'SAFMR\n1BR':'1BR',
                       'SAFMR\n2BR':'2BR',
                       'SAFMR\n3BR':'3BR',
                       'SAFMR\n4BR':'4BR'}, inplace = True)
FMR_df

,zip_code,1BR,2BR,3BR,4BR,HUD Area Code,HUD Metro Fair Market Rent Area Name
0,76437,660,860,1160,1440,METRO10180M10180,"Abilene, TX MSA"
1,76443,660,860,1160,1440,METRO10180M10180,"Abilene, TX MSA"
2,76464,660,860,1160,1440,METRO10180M10180,"Abilene, TX MSA"
3,76469,660,860,1160,1440,METRO10180M10180,"Abilene, TX MSA"
4,79501,700,900,1220,1480,METRO10180M10180,"Abilene, TX MSA"
...,...,...,...,...,...,...,...
27326,85356,770,1010,1430,1720,METRO49740M49740,"Yuma, AZ MSA"
27327,85364,840,1100,1550,1870,METRO49740M49740,"Yuma, AZ MSA"
27328,85365,930,1220,1720,2080,METRO49740M49740,"Yuma, AZ MSA"
27329,85366,810,1070,1510,1820,METRO49740M49740,"Yuma, AZ MSA"


## county_fmr_df

In [7]:
#2. Dataframe of Fair Market Rents by counties (source: ZIP_COUNTY_122021.xlsx)
zip_county_df = pd.read_excel('Data/ZIP_COUNTY_122021.xlsx', dtype=str)
zip_county_df.rename(columns={'county':'fips'}, inplace = True)
zip_county_df = zip_county_df[['zip', 'fips']]
county_FMR_df = pd.read_excel('Data/FY23_FMRs.xlsx', dtype=str)
county_FMR_df.rename(columns={'hud_area_code':'HUD Area Code',
                              'fmr_1':'1BR',
                              'fmr_2':'2BR',
                              'fmr_3':'3BR', 
                              'fmr_4':'4BR'}, inplace = True)
county_FMR_df =county_FMR_df[['fips',
                              '1BR',
                              '2BR',
                              '3BR', 
                              '4BR',
                              'HUD Area Code', 
                              'countyname']]
county_FMR_df['fips'] = county_FMR_df['fips'].str[:5]
county_FMR_df = pd.merge(zip_county_df, county_FMR_df, on='fips', how='left')
county_FMR_df

,zip,fips,1BR,2BR,3BR,4BR,HUD Area Code,countyname
0,00683,72125,417,471,618,705,METRO41900M41900,San Germán Municipio
1,00683,72079,417,471,618,705,METRO41900M41900,Lajas Municipio
2,00683,72023,417,471,618,705,METRO41900M41900,Cabo Rojo Municipio
3,00683,72097,449,507,617,781,METRO32420M32420,Mayagüez Municipio
4,00683,72121,417,471,618,705,METRO41900M41900,Sabana Grande Municipio
...,...,...,...,...,...,...,...,...
123005,60684,17031,1255,1440,1827,2172,METRO16980M16980,Cook County
123006,33945,12071,1141,1451,1896,2157,METRO15980M15980,Lee County
123007,78144,48255,778,879,1234,1340,NCNTY48255N48255,Karnes County
123008,12257,36001,1079,1313,1598,1764,METRO10580M10580,Albany County


## Lookup functions

In [8]:
def lookup_bah_rate(zip,dependent_status, rank):
    BAH = BAH_df[(BAH_df.zip_code == str(zip))&(BAH_df.dep_status == dependent_status)][rank].values[0] #look up BAH rate by zip code and rank
    return BAH 

In [9]:
def lookup_fmr(zip_code, br_number):
    try:
        FMR1 = FMR_df.loc[zip_code, '1BR']
        FMR2 = FMR_df.loc[zip_code, '2BR']
        FMR3 = FMR_df.loc[zip_code, '3BR']
        FMR4 = FMR_df.loc[zip_code, '4BR']
    #If FMR not included by zip code, use county FMR
    except(ValueError):
        FMR1 = county_FMR_df.loc[zip_code, '1BR']
        FMR2 = county_FMR_df.loc[zip_code, '2BR']
        FMR3 = county_FMR_df.loc[zip_code, '3BR']
        FMR4 = county_FMR_df.loc[zip_code, '4BR']
        
    #If FMR not found for zipcode in counties, use national average (https://www.ushousingdata.com/fair-market-rents)
    except:
        FMR1 = 880
        FMR2 = 1096
        FMR3 = 1416
        FMR4 = 1624
    if br_number == 1:
        FMR = FMR1
    elif br_number == 2:
        FMR = FMR2 
    elif br_number == 3:
        FMR = FMR3 
    elif br_number == 4:
        FMR = FMR4 
    return int(FMR)

In [10]:
#Test with input(one zip code) Known value for 93943: $4390 for 3 B
zip_code = 20002
dependent_status = 'W'
rank = 'O03'
br_number = 3
#verify
#lookup_bah_rate(zip_code, dependent_status, rank)
lookup_fmr(zip_code, br_number)

1650

In [11]:
#Test with many input(all zip codes)
#zip_df = pd.read_csv('sorted_zipmha23.txt',sep=" ", header=None,names=["zip_code", "MHA"])
#zip_codes = sorted(list(zip_df['zip_code'].unique()))
#for zips in zip_codes:
#print(zips)
#print(affordability_rating(zips, 'W', 'O04'))
#print(lookup_bah_rate(zips,dependent_status, rank))
#print(lookup_fmr(zips, br_number))

In [12]:
#dictionary for bedroom housing standards(# of bedrms) by rank, dep status
BAH_WD_standards_dict = {'EO1': 2, 
                         'EO2': 2, 
                         'EO3': 2, 
                         'EO4': 2, 
                         'EO5': 2, 
                         'EO6': 3, 
                         'EO7': 3, 
                         'EO8': 3, 
                         'EO9': 3,
                         'WO1': 3,
                         'WO2': 3,
                         'WO3': 3,
                         'WO4': 3,
                         'WO5': 3, 
                         'O01E': 3, 
                         'O02E': 3, 
                         'O03E': 3,
                         'O01': 2,
                         'O02': 2,
                         'O03': 3,
                         'O04': 3,
                         'O05': 4,
                         'O06': 4,
                         'O07': 4,
                         'O08': 4,
                         'O09': 4, 
                         'O010': 4}
BAH_WOD_standards_dict = {'EO1': 1, 
                          'EO2': 1, 
                          'EO3': 1, 
                          'EO4': 1, 
                          'EO5': 1, 
                          'EO6': 2, 
                          'EO7': 2, 
                          'EO8': 2, 
                          'EO9': 2,
                          'WO1': 2,
                          'WO2': 2,
                          'WO3': 2,
                          'WO4': 3,
                          'WO5': 3, 
                          'O01E': 2, 
                          'O02E': 2, 
                          'O03E': 3,
                          'O01': 2,
                          'O02': 2,
                          'O03': 2,
                          'O04': 3,
                          'O05': 3,
                          'O06': 3,
                          'O07': 3,
                          'O08': 3,
                          'O09': 3, 
                          'O010': 3}
#Source: BAH Primer: https://media.defense.gov/2022/Jun/23/2003023204/-1/-1/0/BAH-PRIMER.PDF

In [13]:
#Score: by zip, keep rank/dep status fixed, score is a percentage (BAH/FMR)
def affordability_rating(zip_code,dependent_status, rank): 
    #BAH Primer Standards for house size (bedroom number) based on rank/dep. status
    if dependent_status == 'W':
        br_number = BAH_WD_standards_dict[rank]
    if dependent_status == 'WO':
        br_number = BAH_WOD_standards_dict[rank]
    BAH = lookup_bah_rate(zip_code,dependent_status, rank)
    FMR = lookup_fmr(zip_code, br_number)
    percentage_covered = BAH/FMR
    return percentage_covered

## Objective: Affordability Metric for Zip Based on Rank, Dependency Status, Rank, Housing Standards

In [14]:
#verify
lookup_bah_rate(20007, 'W', 'O04')

3654.0

In [15]:
#verify
lookup_fmr(20007, 3)

1340

In [16]:
#verify
affordability_rating(20007, 'W', 'O04')

2.726865671641791

## Add Rent Affordability Column to project_zips

In [20]:
#Take zip codes from project_zips
zip_codes = sorted(list(project_zips['Zip_Code'].unique()))
len(zip_codes)

1432

In [24]:
rent = list()
for zip in zip_codes:
    rent.append(affordability_rating(int(zip),'W','O04'))
project_zips['Rent Affordability'] = rent
project_zips

,Name,Description,geometry,Zip_Code,Rent Affordability
0,<at><openparen>20001<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-77.02758 38.90964 0.00000, -77.02...",20001,2.502740
1,<at><openparen>20002<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-77.01217 38.89209 0.00000, -77.01...",20002,2.214545
2,<at><openparen>20003<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-77.01402 38.88236 0.00000, -77.01...",20003,2.255556
3,<at><openparen>20004<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-77.03365 38.89735 0.00000, -77.03...",20004,2.726866
4,<at><openparen>20005<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-77.03654 38.90252 0.00000, -77.03...",20005,2.726866
...,...,...,...,...,...
1427,<at><openparen>24646<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-82.07159 37.10896 0.00000, -82.06...",24646,1.519835
1428,<at><openparen>24649<closeparen>,<center><table><tr><th colspan='2' align='cent...,"MULTIPOLYGON Z (((-81.91699 37.03371 0.00000, ...",24649,1.403817
1429,<at><openparen>24651<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-81.78055 36.95863 0.00000, -81.77...",24651,1.234228
1430,<at><openparen>24656<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-82.22942 37.20668 0.00000, -82.22...",24656,1.820792


In [23]:
affordability_rating(22046, 'W','O04')

1.74

In [39]:
dependent_status = 'W' #user input
rank = 'O04' #user input
project_zips['Affordability Rating'] = project_zips.apply(lambda x: affordability_rating(x.Zip_Code, 'W', rank), axis=0)
project_zips

AttributeError: 'Series' object has no attribute 'Zip_Code'

In [30]:
project_zips[project_zips['Zip_Code'] == '22046']

,Name,Description,geometry,Zip Code,Rent Affordability,Zip_Code,Rent_Affordability
591,<at><openparen>22046<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-77.20815 38.88843 0.00000, -77.20...",22046,2.580508,22046,0.9


In [27]:
affordability_rating(22046, 'W','O04')

1.74